# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
%pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

Note: you may need to restart the kernel to use updated packages.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [54]:
# Data loading functions. Uncomment the one you want to use
from worcliver.load_data import load_data
import pandas as pd
import numpy as np
import matplotlib as plt

data = load_data()
df = pd.DataFrame(data)

'''
These 2 lines add a duplicate sample in the DataFrame, you can than observe what the effect is 
in the printed sentence. If you'd like to see it indeed works, please uncomment these lines.
'''
# row = df.iloc[0].copy()
# df = df.append(row, ignore_index=True)

'''
In the lines below we first drop all duplicate entries, whether it is a duplicate row or a column. 
Additionally, the number of duplicates will be returned to the user.
'''

df_f = df.drop_duplicates()
print(f'The number of samples (after removal of duplicates): {len(df_f.index)}, the number of duplicates that had to be removed: {len(df.index)-len(df_f.index)}')
print(f'The number of columns (after removal of duplicates): {len(df_f.columns)}, the number of duplicates that had to be removed: {len(df.columns)-len(df_f.columns)}')


The number of samples (after removal of duplicates): 186, the number of duplicates that have been removed: 0
The number of columns (after removal of duplicates): 494, the number of duplicates that have been removed: 0


# Data clean-up
In this section double entries, such as repeated features and samples, will be removed from the dataset.